## In dit script gaan we het CCC model en het DCC model schatten met normaalverdeling

Om de cel hier onder te runnen moet er een package geinstalleerd worden via anaconda prompt. Deze package zorgt ervoor dat we functies uit andere scripts kunnen gerbuiken in dit script.
<break>
Tik het volgende in anaconda prompt:
<br \>
**pip install ipynb**


In [1]:
%%capture
import pandas as pd
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from ipynb.fs.full.Dataset import getdata
from ipynb.fs.full.Dataset import getreturns
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

Om de GARCH modellen te schatten heb ik mijn in-sample dataset nodig. Mijn in-sample dataset is vanaf 2010 tot en met 2015. Hier gebruik ik een functie die is geschreven in een ander script

In [2]:
df = getreturns()
mInSampleReturns = df.loc[:"2015"]


Om de CCC de DCC modellen te schatten kan de log-likelihood opgedeeld worden in 2 delen.
- Schat de GARCH(1,1) modellen los van elkaar.
- schat daarna de correlatie term

<break>
De multivariate normal distribution van de GARCH modellen zien er als volgt uit:

$$-\frac{1}{2}\sum_{t=2}^{T}(n\log(2\pi)+2\log|D_t|+y_t'D_t^{-1}D_t^{-1}y_t)$$
Dit is gelijk aan de som van de individuele GARCH modellen.
<break>
In de cel hieronder is een functie geschreven om een losse GARCH(1,1) model te schatten. De parameters die we hierbij moeten schatten zijn:

- [$\omega,\alpha,\beta$]

In [3]:
def Parameters_GARCH_Normal(vReturns):
    dOmega = 0.1
    dAlpha = 0.1
    dBeta = 0.8    
    vTheta = np.array([dOmega, dAlpha, dBeta])
    
    def LL_GARCH_Normal(vTheta,returns):
        dOmega = vTheta[0]
        dAlpha = vTheta[1]
        dBeta  = vTheta[2]
        iT=len(returns)
        vH=np.zeros(iT)
        
        for t in range(iT):
            if t == 0:
                vH[t] = np.var(returns) 
            else:
                vH[t] = dOmega + dAlpha*returns[t-1]**2 + dBeta * vH[t-1]    
        
        vLogPdf = -0.5 * np.log( 2 * np.pi * vH[1:] ) - 0.5 * ( returns[1:]**2 / vH[1:] )
        return -np.sum(vLogPdf)
    
    def Optimizer(returns, initials, function, bnds):
        result = minimize(function, initials, args=(returns), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0, 1), (0, 1), (0, 1))
    result=Optimizer(vReturns, vTheta, LL_GARCH_Normal, bounds)
    return result.x, -result.fun, result.success

Hier wordt de eerste parameters en de log-likelihood van een losse GARCH model verkregen van de CAC40. De volgorde van de parameters is:
- [$\omega,\alpha,\beta$]

In [4]:
(parameter1_N,likelihood1_N,success1_N)=Parameters_GARCH_Normal(np.array(mInSampleReturns.iloc[:,0]))
print(parameter1_N)
print(likelihood1_N)
print(success1_N)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2516.193369258651
            Iterations: 12
            Function evaluations: 68
            Gradient evaluations: 12
[ 0.05195353  0.09350152  0.87921127]
-2516.193369258651
True


Hier wordt de eerste parameters en de log-likelihood van een losse GARCH model verkregen van de AEX. De volgorde van de parameters is:
- [$\omega,\alpha,\beta$]

In [5]:
(parameter2_N,likelihood2_N,success2_N)=Parameters_GARCH_Normal(np.array(mInSampleReturns.iloc[:,1]))
print(parameter2_N)
print(likelihood2_N)
print(success2_N)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2238.592048513497
            Iterations: 12
            Function evaluations: 69
            Gradient evaluations: 12
[ 0.03347424  0.09929269  0.8768056 ]
-2238.592048513497
True


#### In de functie Hieronder wordt voor de $\sigma_t^2$ van de losse garch modellen verkregen. Deze zijn nodig als we de log-likelihood van de correlatie term willen berekenen

In [6]:
def ComputeSigma2GARCH(vTheta1_N,vTheta2_N,mReturns):
    iT = mReturns.shape[1]
    iDimension = mReturns.shape[0]
    mH = np.zeros((iDimension,iT))
    dOmega1 = vTheta1_N[0]
    dOmega2 = vTheta2_N[0]
    dAlpha1 = vTheta1_N[1]
    dAlpha2 = vTheta2_N[1]
    dBeta1 = vTheta1_N[2]
    dBeta2 = vTheta2_N[2]
    for t in range(iT):
        if t==0:
            mH[0,t]=np.var(mReturns[0,:])
            mH[1,t]=np.var(mReturns[1,:])
        else:
            mH[0,t]=dOmega1 + dAlpha1 * mReturns[0,t-1]**2 + dBeta1 * mH[0,t-1]
            mH[1,t]=dOmega2 + dAlpha2 * mReturns[1,t-1]**2 + dBeta2 * mH[1,t-1]
    return mH

In [7]:
mSigma2=ComputeSigma2GARCH(parameter1_N,parameter2_N,np.array(mInSampleReturns).T)

Hieronder maximaliseren we de log-likelihood van de correlatie term van het CCC model. 
De log-likelihood van de correlatie term ziet er als volgt uit:
$$-\frac{1}{2}\sum_{t=2}^{T}(\log|R|+\epsilon_t'R^{-1}\epsilon_t-\epsilon_t'\epsilon_t)$$
- waarbij R is gedefinieerd als:
$$R=\begin{pmatrix} 1 & \rho_{12} \\ \rho_{12} & 1 \end{pmatrix}$$
- $\epsilon_t$ als:
$$\epsilon_t=\begin{pmatrix} \epsilon_{1t} \\ \epsilon_{2t}\end{pmatrix}$$
- $\epsilon_{it}$ is verkregen door:
$$\epsilon_{it}=\frac{y_{it}}{\sigma_{it}}$$
De log-likelihood willen we maximaliseren wrt $\rho_{12}$

In [8]:
def Parameters_Correlation_NormalCCC(mReturns,mH):
    dRho12 = 0
    vTheta = np.array([dRho12])
    
    def LL_Correlation_NormalCCC(vTheta,mReturns,mH):
        dRho12 = vTheta[0]
        iT = mReturns.shape[1]
        iDimension = len(mReturns)
        mEpsilon = mReturns/np.sqrt(mH)
        mR = np.ones((iDimension,iDimension))
        mR[1,0] = dRho12
        mR[0,1] = dRho12
        
        dSum=0
        
        for t in range(1,iT):
            vEpsilon_t = mEpsilon[:,t]
            vEpsilon_t = vEpsilon_t.reshape((iDimension,1))
            dLogLikelihood = -0.5 * (np.log(det(mR))+ vEpsilon_t.T @ inv(mR) @ vEpsilon_t - vEpsilon_t.T @ vEpsilon_t)
            dSum += np.asscalar(dLogLikelihood)
  
        return -dSum
    
    def Optimizer(returns, mH, initials, function, bnds):
        result = minimize(function, initials, args=(returns,mH), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((-1, 0.9999999),)
    result=Optimizer(mReturns, mH, vTheta, LL_Correlation_NormalCCC, bounds)
    return result.x, -result.fun, result.success

In [9]:
(dRho,LL_Correlation_N,success_Corr_N)=Parameters_Correlation_NormalCCC(np.array(mInSampleReturns).T,mSigma2)
print(dRho)
print(LL_Correlation_N)
print(success_Corr_N)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1489.8786032511491
            Iterations: 29
            Function evaluations: 108
            Gradient evaluations: 28
[ 0.92608184]
1489.8786032511491
True


Hier gaan we de correlatie term van het DCC model schatten. De log-likelihood ziet er als volgt uit:
$$-\frac{1}{2}\sum_{t=2}^{T}(\log|R_t|+\epsilon_t'R_t^{-1}\epsilon_t-\epsilon_t'\epsilon_t)$$
- Waarbij $R_t$ is gedefinieerd als:
$$ R_t=\text{diag}(Q_t)^{\frac{-1}{2}}Q_t\text{diag}(Q_t)^{\frac{-1}{2}}$$
- Q_t is gedefinieerd als:
$$Q_t=(1-A-B)S + A\epsilon_{t-1}\epsilon_{t-1}' + BQ_{t-1}$$
-S is gedefinieerd als:
$$S=\frac{1}{T}\sum_{t=1}^{T}\epsilon_t\epsilon_t'$$
verder gebruik ik als initialisatie van $Q_1=S$
<break>
- De $diag(Q_t)^\frac{-1}{2}$ is een diagonaal matrix waarbij de elementen op de diagonaal matrix hetzelfde zijn als die van $Q_t$ alleen dan nog in de wortel genomen
- In andere woorden:
$$diag(Q_t)^{-\frac{1}{2}}=\begin{pmatrix} \sqrt(q_{11t}) & 0 \\ 0 & \sqrt(q_{22t}) \end{pmatrix}^{-1}, Q_t=\begin{pmatrix} q_{11t} & q_{12t} \\ q_{21t} & q_{22t} \end{pmatrix} $$
- De log-likelihood wordt gemaximaliseerd wrt A en B waarbij A en B getallen zijn en ze moeten positief zijn

In [10]:
def Parameters_Correlation_NormalDCC(mReturns,mH):
    dA = 0.1
    dB = 0.7
    vTheta = np.array([dA,dB])
    
    def LL_Correlation_NormalDCC(vTheta,mReturns,mH):
        dA = vTheta[0]
        dB = vTheta[1]
        iT = mReturns.shape[1]
        iDimension = len(mReturns)
        mEpsilon = mReturns/np.sqrt(mH)

        mS_sum = np.zeros((iDimension,iDimension))
        for i in range(iT):
            vEpsilon_t = mEpsilon[:,i]
            vEpsilon_t = vEpsilon_t.reshape((iDimension,1))
            mS_sum += vEpsilon_t @ vEpsilon_t.T
        
        mS = mS_sum / iT
        mQ_old=np.copy(mS)
        
        dSum=0
        for t in range(1,iT):
            vEpsilon_lag=mEpsilon[:,t-1]
            vEpsilon_lag=vEpsilon_lag.reshape((iDimension,1))
            vEpsilon=mEpsilon[:,t]
            vEpsilon=vEpsilon.reshape((iDimension,1))
            
            mQ = (1-dA-dB) * mS + dA * vEpsilon_lag @ vEpsilon_lag.T + dB * mQ_old 
            mQ_diagonal = np.sqrt(np.diagonal(mQ))
            mR_t= inv(np.diag(mQ_diagonal)) @ mQ @ inv(np.diag(mQ_diagonal)) 
            
            dLogLikelihood = -0.5 * (np.log(det(mR_t))+ vEpsilon.T @ inv(mR_t) @ vEpsilon - vEpsilon.T @ vEpsilon)
            dSum += np.asscalar(dLogLikelihood)
            mQ_old=mQ   
        return -dSum
    
    def Optimizer(returns, mH, initials, function, bnds):
        result = minimize(function, initials, args=(returns,mH), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0.0001, 0.9999), (0.0001,0.9999))
    result=Optimizer(mReturns, mH, vTheta, LL_Correlation_NormalDCC, bounds)
    return result.x, -result.fun, result.success

In [11]:
(vParametersDCC,LL_Correlation_DCC,success_Corr_DCC)=Parameters_Correlation_NormalDCC(np.array(mInSampleReturns).T,mSigma2)
print(vParametersDCC)
print(LL_Correlation_DCC)
print(success_Corr_DCC)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1536.4081115935837
            Iterations: 13
            Function evaluations: 63
            Gradient evaluations: 13
[ 0.06556189  0.88592729]
1536.4081115935837
True


Hier gaan we de correlatie term van het DCC model schatten. De log-likelihood ziet er als volgt uit:
$$-\frac{1}{2}\sum_{t=2}^{T}(\log|R_t|+\epsilon_t'R_t^{-1}\epsilon_t-\epsilon_t'\epsilon_t)$$
- Waarbij $R_t$ is gedefinieerd als:
$$ R_t=\text{diag}(Q_t)^{\frac{-1}{2}}Q_t\text{diag}(Q_t)^{\frac{-1}{2}}$$
- Q_t is gedefinieerd als:
$$Q_t=(1-A-B)S -G\bar{N} + A\epsilon_{t-1}\epsilon_{t-1}' + Gn_{t-1}n_{t-1}' + BQ_{t-1}$$
- S is gedefinieerd als:
$$S=\frac{1}{T}\sum_{t=1}^{T}\epsilon_t\epsilon_t'$$
verder gebruik ik als initialisatie voor $Q_1=S$
<break>
- n_{t-1} is gedefinieerd als: 
$$
n_{t-1}=\mathbb{1}(\epsilon_{t-1})\odot \epsilon_{t-1}
\\
\mathbb{1}=\begin{cases}
1&\text{if $\epsilon_{t-1}<0$}\\
0&\text{otherwise}\\
\end{cases}
$$
- Hierbij is $\bar{N}$ gedefinieerd als:
$$\bar{N}=\frac{1}{T}\sum_{t=1}^{T}n_tn_t'
$$
Verder gebruiken we als initialisatie $n_1n_1'=\bar{N}$
- De $diag(Q_t)^\frac{-1}{2}$ is een diagonaal matrix waarbij de elementen op de diagonaal matrix hetzelfde zijn als die van $Q_t$ alleen dan nog in de wortel genomen
- In andere woorden:
$$diag(Q_t)^{-\frac{1}{2}}=\begin{pmatrix} \sqrt(q_{11t}) & 0 \\ 0 & \sqrt(q_{22t}) \end{pmatrix}^{-1}, Q_t=\begin{pmatrix} q_{11t} & q_{12t} \\ q_{21t} & q_{22t} \end{pmatrix} $$
- De log-likelihood wordt gemaximaliseerd wrt A, B, G waarbij A, B en G getallen zijn en ze moeten allemaal positief zijn


In [12]:
def Parameters_Correlation_NormalADCC(mReturns,mH):
    dA = 0.1
    dB = 0.7
    dG = 0.1
    vTheta = np.array([dA,dB,dG])
    
    def LL_Correlation_NormalADCC(vTheta,mReturns,mH):
        dA = vTheta[0]
        dB = vTheta[1]
        dG = vTheta[2]
        iT = mReturns.shape[1]
        iDimension = len(mReturns)
        mEpsilon = mReturns/np.sqrt(mH)
        mZeros = np.zeros(mReturns.shape)
        mDummy=(mEpsilon<mZeros)+mZeros
        mN=mEpsilon*mDummy
        
        mS_sum = np.zeros((iDimension,iDimension))
        mN_sum = np.zeros((iDimension,iDimension))
        for i in range(iT):
            vN_t=mN[:,i]
            vN_t=vN_t.reshape((iDimension,1))
            mN_sum += vN_t @ vN_t.T
            vEpsilon_t = mEpsilon[:,i]
            vEpsilon_t = vEpsilon_t.reshape((iDimension,1))
            mS_sum += vEpsilon_t @ vEpsilon_t.T
        
        mS = mS_sum / iT
        mN_bar=mN_sum / iT
        mQ_old=np.copy(mS)
        
        dSum=0
        for t in range(1,iT):
            vEpsilon_lag=mEpsilon[:,t-1]
            vEpsilon_lag=vEpsilon_lag.reshape((iDimension,1))
            vEpsilon=mEpsilon[:,t]
            vEpsilon=vEpsilon.reshape((iDimension,1))
            vN_lag=mN[:,t-1]
            vN_lag=vN_lag.reshape((iDimension,1))
            
            mQ = (1-dA-dB) * mS - dG * mN_bar + dA * vEpsilon_lag @ vEpsilon_lag.T + dG * vN_lag @ vN_lag.T + dB * mQ_old 
            mQ_diagonal = np.sqrt(np.diagonal(mQ))
            mR_t= inv(np.diag(mQ_diagonal)) @ mQ @ inv(np.diag(mQ_diagonal)) 
            
            dLogLikelihood = -0.5 * (np.log(det(mR_t))+ vEpsilon.T @ inv(mR_t) @ vEpsilon - vEpsilon.T @ vEpsilon)
            dSum += np.asscalar(dLogLikelihood)
            mQ_old=mQ   
        return -dSum
    
    def Optimizer(returns, mH, initials, function, bnds):
        result = minimize(function, initials, args=(returns,mH), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0.0001, 0.9999), (0.0001,0.9999), (0.00001,0.9999))
    result=Optimizer(mReturns, mH, vTheta, LL_Correlation_NormalADCC, bounds)
    return result.x, -result.fun, result.success

In [13]:
(vParamtersADCC,LL_Correlation_ADCC,success_Corr_ADCC)=Parameters_Correlation_NormalADCC(np.array(mInSampleReturns).T,mSigma2)
print(vParamtersADCC)
print(LL_Correlation_ADCC)
print(success_Corr_ADCC)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1536.7784450693255
            Iterations: 13
            Function evaluations: 80
            Gradient evaluations: 13
[ 0.05261769  0.89170114  0.01877666]
1536.7784450693255
True
